# Univariate

In [1]:
using LinearAlgebra
using DynamicPolynomials
using MultivariateMoments
using JuMP
using MacaulayMatrix

Consider the following example:

In [2]:
@polyvar x
p = 3x^4 + 8x^3 - 6x^2 + 24x + 1
q = differentiate(p, x)

24 - 12x + 24x² + 12x³

Let's solve that system:

In [3]:
sols = solve_system([q], column_maxdegree = 4)
nothing # hide

[ Info: Added 1 rows to complete columns up to degree 3
[ Info: Nullspace of dimensions (4, 3) computed from Macaulay matrix of dimension (1, 4) in 5.5132e-5 seconds.
[ Info: Found 1 real solution


The real roots are:

In [4]:
sols

1-element Vector{Vector{Float64}}:
 [-2.658967081916994]

With PSD Hankel matrix trick no complex root!

In [5]:
import SCS
solver = SCS.Optimizer
psd_hankel([q], solver, 4)

[ Info: Nullspace of dimensions (5, 3) computed from Macaulay matrix of dimension (2, 5) in 6.0353e-5 seconds.
------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3, constraints m: 7
cones: 	  z: primal zero / dual free vars: 1
	  s: psd vars: 6, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 21, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.3

1-element Vector{Vector{Float64}}:
 [-2.6593904239378494]

What happened there ?
First, we computed the MacaulayMatrix matrix nullspace

In [6]:
Z = nullspace(macaulay([q], 4))

[ Info: Nullspace of dimensions (5, 3) computed from Macaulay matrix of dimension (2, 5) in 4.5555e-5 seconds.


MultivariateMoments.MacaulayNullspace{Float64, LinearAlgebra.Adjoint{Float64, Matrix{Float64}}, MultivariateBases.MonomialBasis{DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}}([-0.5648877062932673 -0.5061871608167463 -0.08949732306804505; 0.35309169447067507 -0.5869837085383489 -0.3054118223094746; … ; -0.005487708235674751 0.6097339283112615 -0.15500888702836046; 0.048969435176441645 -0.13767209715388368 0.9351372741031582], MonomialBasis([1, x, x², x³, x⁴]), 5.438959822042073e-16)

The PSD hankel matrix we find is:

In [7]:
M = moment_matrix(Z, solver, 2)

------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3, constraints m: 7
cones: 	  z: primal zero / dual free vars: 1
	  s: psd vars: 6, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 21, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.31e+00  2.08e-01  5.54e-01  2.77e-01  1.00e-01  1.58e-04 
   250| 5.50e-01  1.41e-05  2.91e-04  1.45e-04  1.48e-

MomentMatrix with row/column basis:
 MonomialBasis([1, x, x^2])
And entries in a 3×3 MultivariateMoments.SymMatrix{Float64}:
  1.0000000181608577   -2.6527137858823586    7.054717498400912
 -2.6527137858823586    7.054717498400912   -18.762148819005894
  7.054717498400912   -18.762148819005894    49.88444270817741

From which we get:

In [8]:
atomic_measure(M, 1e-6)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*